In [1]:
import pandas as pd
from sklearn import metrics
from scipy.sparse import hstack
import joblib

pd.set_option("display.max_colwidth", None)

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Read the pre-processed data

In [3]:
data_path = '/content/drive/MyDrive/dialect_classification/training/preprocessed_data_2.csv'

base_path = '/content/drive/MyDrive/dialect_classification/'
df = pd.read_csv(base_path+'training/preprocessed_data_2.csv', lineterminator='\n')
df

,text,label
0,قليلين ادب ومنافقين لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المراه من رده فعلهم,LY
1,الليبيين متقلبين بس بالنسبه ليا انا ميليشياوي زمان وتوه,LY
2,كل تانيه شاب ليبي بيرتاح لبنت مختلفه ويلاحظ انها غير كل البنات وبيحس كانه يعرفها من زمان بعدين يتزوج وحده منهن وممكن اثنين ولاثلاثه وتنقلب الرومانسيه لعياط وشياط وتهزيب ذي اند,LY
3,رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلا ثانيا شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات يعني اليتيمه متستحقش تتزوج وثالثا ليش البنت هي بس لازم ادير الف حساب للراجل هي متستحقش يندارلها الف حساب ولا هي عبده,LY
4,شكلك متعقده علشان الراجل لي تحبيه ازوج بنت يتيمه ولا بنت معندهش خوت هدي اعصابك وفكينا من التخلف امتاعك,LY
...,...,...
147714,الناس دي بتنفخ في قربه مقدوده بالدارجي كده البلد دي لو ما الكيزان ديل راحو مافي حياه شريفه في البلد دي لان سبب الفساد والمصاءب الموجوده التطلع لسودان افضل,SD
147715,انت عايش وين بره السودان ولا شنو ماشايف البحصل دا,SD
147716,مااحرم نفسي ميسي حريف ولعاب برضو مدريدي وافتخر,SD
147717,ياخي ديل ماخلو للشيطان وابليس شي لكن يروحو وين من ربنا ونحن وراك وراك ياكوز ياعفن الايام معدوده خلاص تعرف علي الكيزان,SD


# Modelling

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

## check sample of the data

In [5]:
df_sample = df.sample(n=100, random_state=1)

In [6]:
df_sample

,text,label
83789,كذبه مشهوره للمصريين هجيلك بعد عشر دقايق,EG
5641,تو في واحد زعلان علي وحده يقعد يقول فكو طيري و يلحن فيها انا ضيقلي علمي الفيديو و الوهم اللي صاير,LY
73658,الشباب دماغهم بقت فاضيه اي السبب للعلم انا بردو دماغي كانت فاضيه,EG
27149,خلاص امشي الصبح وبعدين زي ما قالولك العيد ميلاد يتكرر بس التخرج لا,LY
125466,عيش كل يوم بيوموا ووكل امرك لربنا,LB
...,...,...
116366,اي بزفتا طالعين كذا مره النيه كتير طيبه,LB
59721,هنلاقيها من البقر اللي بينعرو كل كام ساعه في ودننا بالميكروفونات ويقلقونا من النوم ولا من ضراط الموتوسيكلات ولا من مهرجانات التكاتك ولا من انذارات العربيات ولا من العيال اللي بتلعب كوره وتنعر فالشارع الهي تيجي مصيبه بكره الصبح تشيل البلد دي باللي عليها ونرتاح كلنا ونخلص مالقرف ده,EG
103567,مين دول اساسا بجد,EG
100843,لو لقيت حد نايم في الشارع في البرد يرجي الاتصال في اي وقت لتوصيل بطانيه له وليد العدوي السلام عليكم و رحمه الله وبركاته اللي بيعرف عاءله محتاجه ملابس للشتاء وناس محتاجين لبس ايا كان توجد ملابس,EG


## Split the data

In [7]:
X = df['text']
y = df['label']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

## Trying out models

In [8]:
# character n-gram
cv_char = CountVectorizer(ngram_range=(3,4), analyzer='char')
# X_char = cv_char.fit_transform(df_sample.text)
# print("Character n-grams matrix:")
# print(X_char.toarray())

In [9]:
model = LinearSVC()
pipe = make_pipeline(cv_char, model)
pipe.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(analyzer='char', ngram_range=(3, 4))),
                ('linearsvc', LinearSVC())])

In [10]:
def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

In [11]:
print('Only character 3-4-gram results:')
print_report(pipe, x_test, y_test)

Only character 3-4-gram results:
              precision    recall  f1-score   support

          EG       0.85      0.87      0.86     11527
          LB       0.83      0.82      0.82      5523
          LY       0.78      0.78      0.78      7299
          MA       0.73      0.69      0.71      2308
          SD       0.68      0.65      0.67      2887

    accuracy                           0.80     29544
   macro avg       0.77      0.76      0.77     29544
weighted avg       0.80      0.80      0.80     29544

accuracy: 0.804


Only character 3-4-gram results:

              precision    recall  f1-score   support

          EG       0.85      0.87      0.86     11527
          LB       0.83      0.82      0.82      5523
          LY       0.78      0.78      0.78      7299
          MA       0.73      0.69      0.71      2308
          SD       0.68      0.65      0.67      2887

     accuracy                          0.80     29544
     macro avg     0.77      0.76      0.77     29544
    weighted avg   0.80      0.80      0.80     29544

accuracy: 0.804

In [12]:
# word N-gram
cv_word = CountVectorizer(ngram_range=(2, 6), analyzer='word')
model = LinearSVC()
pipe = make_pipeline(cv_word, model)
pipe.fit(x_train, y_train)
print('Only word 2-6-gram results:')
print_report(pipe, x_test, y_test)

Only word 2-6-gram results:
              precision    recall  f1-score   support

          EG       0.58      0.91      0.71     11527
          LB       0.81      0.54      0.65      5523
          LY       0.66      0.56      0.61      7299
          MA       0.88      0.21      0.34      2308
          SD       0.79      0.25      0.38      2887

    accuracy                           0.64     29544
   macro avg       0.74      0.50      0.54     29544
weighted avg       0.69      0.64      0.61     29544

accuracy: 0.636


Only word 2-6-gram results:

              precision    recall  f1-score   support

          EG       0.58      0.91      0.71     11527
          LB       0.81      0.54      0.65      5523
          LY       0.66      0.56      0.61      7299
          MA       0.88      0.21      0.34      2308
          SD       0.79      0.25      0.38      2887

    accuracy                           0.64     29544
    macro avg      0.74      0.50      0.54     29544
    weighted avg   0.69      0.64      0.61     29544

accuracy: 0.636

In [13]:
# 3-7 char N-gram
cv_char_2 = CountVectorizer(ngram_range=(3, 7), analyzer='char')
model = LinearSVC()
pipe = make_pipeline(cv_char_2, model)
pipe.fit(x_train, y_train)
print('Only character 3-7-gram results:')
print_report(pipe, x_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Only character 3-7-gram results:
              precision    recall  f1-score   support

          EG       0.85      0.91      0.88     11527
          LB       0.86      0.86      0.86      5523
          LY       0.81      0.82      0.82      7299
          MA       0.82      0.69      0.75      2308
          SD       0.78      0.66      0.71      2887

    accuracy                           0.84     29544
   macro avg       0.83      0.79      0.80     29544
weighted avg       0.83      0.84      0.83     29544

accuracy: 0.836


Only character 3-7-gram results:

              precision    recall  f1-score   support

          EG       0.85      0.91      0.88     11527
          LB       0.86      0.86      0.86      5523
          LY       0.81      0.82      0.82      7299
          MA       0.82      0.69      0.75      2308
          SD       0.78      0.66      0.71      2887

    accuracy                           0.84     29544
    macro avg      0.83      0.79      0.80     29544
    weighted avg   0.83      0.84      0.83     29544

accuracy: 0.836

In [14]:
import re

def words_and_char_bigrams(text):
    words = re.findall(r'\w{3,}', text)
    for w in words:
        yield w
        for i in range(len(w) - 2):
            yield w[i:i+2]

In [15]:
# words and char bigrams
cv_char_word_1 = CountVectorizer(analyzer=words_and_char_bigrams)
model = LinearSVC()
pipe = make_pipeline(cv_char_word_1, model)
pipe.fit(x_train, y_train)
print('Character and word bi-gram results:')
print_report(pipe, x_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Character and word bi-gram results:
              precision    recall  f1-score   support

          EG       0.83      0.89      0.86     11527
          LB       0.83      0.82      0.82      5523
          LY       0.79      0.79      0.79      7299
          MA       0.79      0.66      0.72      2308
          SD       0.71      0.61      0.66      2887

    accuracy                           0.81     29544
   macro avg       0.79      0.75      0.77     29544
weighted avg       0.80      0.81      0.80     29544

accuracy: 0.805


Character word bi-gram results:

              precision    recall  f1-score   support

          EG       0.83      0.89      0.86     11527
          LB       0.83      0.82      0.82      5523
          LY       0.79      0.79      0.79      7299
          MA       0.79      0.66      0.72      2308
          SD       0.71      0.61      0.66      2887

    accuracy                           0.81     29544
    macro avg      0.79      0.75      0.77     29544
    weighted avg   0.80      0.81      0.80     29544

accuracy: 0.805

In [16]:
# words and char bigrams
cv_char_2 = CountVectorizer(ngram_range=(2, 6), analyzer='char')
cv_word_2 = CountVectorizer(ngram_range=(2, 6), analyzer='word')
model = LinearSVC()
x_train_char = cv_char_2.fit_transform(x_train)
x_train_word = cv_word_2.fit_transform(x_train)
x_train_full = hstack((x_train_char, x_train_word))
x_test_char = cv_char_2.transform(x_test)
x_test_word = cv_word_2.transform(x_test)
x_test_full = hstack((x_test_char, x_test_word))
model.fit(x_train_full, y_train)
print('Character and word 2-6-gram results:')
print_report(model, x_test_full, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Character and word 2-6-gram results:
              precision    recall  f1-score   support

          EG       0.86      0.91      0.88     11527
          LB       0.87      0.86      0.86      5523
          LY       0.81      0.83      0.82      7299
          MA       0.83      0.70      0.76      2308
          SD       0.78      0.66      0.72      2887

    accuracy                           0.84     29544
   macro avg       0.83      0.79      0.81     29544
weighted avg       0.84      0.84      0.84     29544

accuracy: 0.840


Character and word 2-6-gram results:

              precision    recall  f1-score   support

          EG       0.86      0.91      0.88     11527
          LB       0.87      0.86      0.86      5523
          LY       0.81      0.83      0.82      7299
          MA       0.83      0.70      0.76      2308
          SD       0.78      0.66      0.72      2887

    accuracy                           0.84     29544
    macro avg      0.83      0.79      0.81     29544
    weighted avg   0.84      0.84      0.84     29544

accuracy: 0.840


In [19]:
filename = 'char_word_model'
joblib.dump(model, filename)
joblib.dump(cv_char_2, filename+'_cv_char')
joblib.dump(cv_word_2, filename+'_cv_word')
# loaded_model = joblib.load(filename)

['char_word_modelcv_word']